In [ ]:
from adaptive_mesh_refiner.adaptive_meshing import AdaptiveMeshRefiner
from src.radiation_algorithm.radiation_algorithm import radiation_algorithm
from efield.efield1 import calculate_electric_magnetic_field_at_point
from efield.efield2 import radiation_intensity_distribution_over_sphere_surface
from efield.efield3 import antenna_directivity_pattern

import numpy as np
import os
import time

In [ ]:
def traitement_de_(filename):
    base_name = os.path.splitext(os.path.basename(filename))[0]
    print(f"Traitement de l'antenne {base_name}")

filename_mesh1_bowtie2 = 'data/antennas_mesh/bowtie2.mat'

In [ ]:
W = 25 / 1000
H = 50 / 1000
wf = 2 / 1000

points = {
    'point_0' : (-W/2, -H/2),
    'point_1' : (-wf/2, 0),
    'point_2' : (-W/2, H/2),
    'point_3' : (W/2, H/2),
    'point_4' : (wf/2, 0),
    'point_5' : (W/2, -H/2)
}

# Declaration des points du contour
vertices = [points[point] for point in points]

# Indices des segments formant le contour
segments = [(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 0), (1, 4)]

# Construire le dictionnaire pour Triangle
bowtie2_antenna = dict(vertices=vertices, segments=segments)


In [ ]:
feed_point_bowtie2 = [0, 0, 0]
voltage_amplitude = 1  # Valeur de l'amplitude égale à 1 volt

In [ ]:
bowtie2_name = "bowtie2"
bowtie2_folder = "data/antennas_mesh/"

# Création de l'objet de raffinage
mesh_bowtie2 = AdaptiveMeshRefiner()

# Étape 1 : Génération de la triangulation initiale avec un niveau moyen
mesh_bowtie2.initial_meshing(bowtie2_antenna, bowtie2_name, bowtie2_folder, refinement_level="ultra")

# show_mesh(points, triangles, feed_point_IFA)  # Change "black" si besoin
mesh_bowtie2.show_mesh(feed_point_bowtie2)

In [ ]:
number_of_refinement = 5  # Nombre maximum d'itérations

tolerance = 1e-2  # Seuil de variation relatif (1%)

previous_impedance = None  # Stocke l'impédance de l'itération précédente
actual_iterations = 0  # Compteur d'itérations effectives

for i in range(number_of_refinement):
    start_time = time.time()
    
    traitement_de_(filename_mesh1_bowtie2)
    impedance, selected_triangles = radiation_algorithm(filename_mesh1_bowtie2, 75e6, feed_point_bowtie2, voltage_amplitude, load_from_matlab=False)
    
    elapsed_time = time.time() - start_time
    print(f"Temps écoulé pour le traitement de l'antenne étudiée : {elapsed_time:.6f} secondes")
    print(f"Impédance actuelle : {impedance}\n")

    actual_iterations += 1
    
    # Vérifier la convergence de l'impédance
    if previous_impedance is not None:
        delta_impedance = np.abs(impedance - previous_impedance) / np.abs(previous_impedance)
        print(f"Variation relative de l'impédance : {delta_impedance:.6f}")

        if delta_impedance < tolerance:
            print("La variation de l'impédance est inférieure au seuil. Arrêt du raffinage.")
            break  # Sortir de la boucle

    previous_impedance = impedance  # Mettre à jour l'impédance précédente

    # Tentative d'affinage du maillage
    try:
        mesh_bowtie2.adaptative_meshing(bowtie2_antenna, selected_triangles, bowtie2_name, bowtie2_folder)
    except ValueError as e:
        print(f"Exception attrapée : {e}")
        break  # Arrêter si une erreur survient

# Affichage du nombre d'itérations utilisées
print(f"Nombre total d'itérations effectuées : {actual_iterations}")
